In [3]:
"""
Pure Python + NumPy + SciPy Bayesian Optimization
Gaussian Process + Expected Improvement + XGBoost
"""

import numpy as np
from scipy.spatial.distance import cdist
from scipy.linalg import cholesky, solve
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import random


# =====================================
# 1. Load dataset
# =====================================
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)


# =====================================
# 2. Gaussian Process Kernel (Matern 5/2)
# =====================================
def matern52_kernel(X1, X2, l=1.0, sigma_f=1.0):
    r = cdist(X1, X2, metric='euclidean')
    sqrt5_r = np.sqrt(5) * r
    return sigma_f**2 * (1 + sqrt5_r + (5/3)*r**2) * np.exp(-sqrt5_r)


# =====================================
# 3. GP Prediction
# =====================================
def gp_predict(X_train, y_train, X_test, l=1.0, sigma_f=1.0, sigma_n=1e-6):
    K = matern52_kernel(X_train, X_train, l, sigma_f) + sigma_n*np.eye(len(X_train))
    K_s = matern52_kernel(X_train, X_test, l, sigma_f)
    K_ss = matern52_kernel(X_test, X_test, l, sigma_f) + 1e-8*np.eye(len(X_test))

    L = cholesky(K, lower=True)
    alpha = solve(L.T, solve(L, y_train))

    mu = K_s.T @ alpha
    v = solve(L, K_s)
    cov = K_ss - v.T @ v
    sigma = np.sqrt(np.maximum(np.diag(cov), 1e-9))
    return mu, sigma


# =====================================
# 4. Expected Improvement (EI)
# =====================================
from scipy.stats import norm

def expected_improvement(mu, sigma, f_best, xi=0.01):
    Z = (f_best - mu - xi) / sigma
    EI = (f_best - mu - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    EI[sigma == 0.0] = 0.0
    return EI


# =====================================
# 5. Objective Function Wrapper for XGBoost
# =====================================
def objective(params):
    (max_depth, lr, n_est, subs, col) = params

    model = XGBRegressor(
        max_depth=int(max_depth),
        learning_rate=float(lr),
        n_estimators=int(n_est),
        subsample=float(subs),
        colsample_bytree=float(col),
        tree_method="hist",
        random_state=42,
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


# =====================================
# 6. Define search space
# =====================================
bounds = np.array([
    [3, 12],        # max_depth
    [0.01, 0.3],    # learning_rate
    [100, 600],     # n_estimators
    [0.5, 1.0],     # subsample
    [0.5, 1.0],     # colsample_bytree
])


# Random sample
def sample_random():
    return np.array([
        np.random.randint(bounds[0][0], bounds[0][1]+1),
        np.random.uniform(bounds[1][0], bounds[1][1]),
        np.random.randint(bounds[2][0], bounds[2][1]+1),
        np.random.uniform(bounds[3][0], bounds[3][1]),
        np.random.uniform(bounds[4][0], bounds[4][1]),
    ])


# =====================================
# 7. Bayesian Optimization Loop
# =====================================
N_INIT = 5
N_ITER = 50

X_obs = []
y_obs = []

# ---- Initial random points
for _ in range(N_INIT):
    x = sample_random()
    y = objective(x)
    X_obs.append(x)
    y_obs.append(y)

X_obs = np.array(X_obs)
y_obs = np.array(y_obs)


for t in range(N_ITER):
    # Candidate pool
    candidates = np.array([sample_random() for _ in range(200)])

    mu, sigma = gp_predict(X_obs, y_obs, candidates)

    f_best = np.min(y_obs)
    EI = expected_improvement(mu, sigma, f_best)

    next_idx = np.argmax(EI)
    next_x = candidates[next_idx]

    y_next = objective(next_x)

    X_obs = np.vstack([X_obs, next_x])
    y_obs = np.append(y_obs, y_next)

    print(f"[BO] Iter {t+1}/{N_ITER} | Best RMSE = {np.min(y_obs):.4f}")


print("\n===== FINAL RESULTS =====")
print("Best RMSE:", np.min(y_obs))


[BO] Iter 1/50 | Best RMSE = 0.4437
[BO] Iter 2/50 | Best RMSE = 0.4437
[BO] Iter 3/50 | Best RMSE = 0.4437
[BO] Iter 4/50 | Best RMSE = 0.4437
[BO] Iter 5/50 | Best RMSE = 0.4437
[BO] Iter 6/50 | Best RMSE = 0.4437
[BO] Iter 7/50 | Best RMSE = 0.4437
[BO] Iter 8/50 | Best RMSE = 0.4437
[BO] Iter 9/50 | Best RMSE = 0.4437
[BO] Iter 10/50 | Best RMSE = 0.4437
[BO] Iter 11/50 | Best RMSE = 0.4437
[BO] Iter 12/50 | Best RMSE = 0.4437
[BO] Iter 13/50 | Best RMSE = 0.4437
[BO] Iter 14/50 | Best RMSE = 0.4437
[BO] Iter 15/50 | Best RMSE = 0.4437
[BO] Iter 16/50 | Best RMSE = 0.4437
[BO] Iter 17/50 | Best RMSE = 0.4437
[BO] Iter 18/50 | Best RMSE = 0.4437
[BO] Iter 19/50 | Best RMSE = 0.4437
[BO] Iter 20/50 | Best RMSE = 0.4437
[BO] Iter 21/50 | Best RMSE = 0.4437
[BO] Iter 22/50 | Best RMSE = 0.4437
[BO] Iter 23/50 | Best RMSE = 0.4437
[BO] Iter 24/50 | Best RMSE = 0.4437
[BO] Iter 25/50 | Best RMSE = 0.4437
[BO] Iter 26/50 | Best RMSE = 0.4437
[BO] Iter 27/50 | Best RMSE = 0.4437
[BO] Iter 